In [1]:
import numpy as np
import os
from torch.utils.data import Dataset
from pointnet_util import farthest_point_sample, pc_normalize

class ModelNetDataLoader(Dataset):
    def __init__(self, root, npoint=2048, split='train', uniform=False, normal_channel=True, cache_size=15000):
        self.root = root
        self.npoints = npoint
        self.uniform = uniform
        self.catfile = os.path.join(self.root, 'modelnet40_shape_names.txt')

        self.cat = [line.rstrip() for line in open(self.catfile)]
        self.classes = dict(zip(self.cat, range(len(self.cat))))
        self.normal_channel = normal_channel

        shape_ids = {}
        shape_ids['train'] = [line.rstrip() for line in open(os.path.join(self.root, 'modelnet40_train.txt'))]
        shape_ids['test'] = [line.rstrip() for line in open(os.path.join(self.root, 'modelnet40_test.txt'))]

        assert (split == 'train' or split == 'test')
        shape_names = ['_'.join(x.split('_')[0:-1]) for x in shape_ids[split]]
        # list of (shape_name, shape_txt_file_path) tuple
        self.datapath = [(shape_names[i], os.path.join(self.root, shape_names[i], shape_ids[split][i]) + '.txt') for i
                         in range(len(shape_ids[split]))]
        print('The size of %s data is %d'%(split,len(self.datapath)))

        self.cache_size = cache_size  # how many data points to cache in memory
        self.cache = {}  # from index to (point_set, cls) tuple

    def __len__(self):
        return len(self.datapath)

    def _get_item(self, index):
        if index in self.cache:
            point_set, cls = self.cache[index]
        else:
            fn = self.datapath[index]
            cls = self.classes[self.datapath[index][0]]
            cls = np.array([cls]).astype(np.int32)
            point_set = np.loadtxt(fn[1], delimiter=',').astype(np.float32)
            if self.uniform:
                point_set = farthest_point_sample(point_set, self.npoints)
            else:
                point_set = point_set[0:self.npoints,:]

            point_set[:, 0:3] = pc_normalize(point_set[:, 0:3])

            if not self.normal_channel:
                point_set = point_set[:, 0:3]

            if len(self.cache) < self.cache_size:
                self.cache[index] = (point_set, cls)

        return point_set, cls

    def __getitem__(self, index):
        return self._get_item(index)

In [5]:
TRAIN_DATASET = ModelNetDataLoader(root='./modelnet40_normal_resampled', npoint=10000, split='train', normal_channel=False)

The size of train data is 9843


In [6]:
TRAIN_DATASET.classes

{'airplane': 0,
 'bathtub': 1,
 'bed': 2,
 'bench': 3,
 'bookshelf': 4,
 'bottle': 5,
 'bowl': 6,
 'car': 7,
 'chair': 8,
 'cone': 9,
 'cup': 10,
 'curtain': 11,
 'desk': 12,
 'door': 13,
 'dresser': 14,
 'flower_pot': 15,
 'glass_box': 16,
 'guitar': 17,
 'keyboard': 18,
 'lamp': 19,
 'laptop': 20,
 'mantel': 21,
 'monitor': 22,
 'night_stand': 23,
 'person': 24,
 'piano': 25,
 'plant': 26,
 'radio': 27,
 'range_hood': 28,
 'sink': 29,
 'sofa': 30,
 'stairs': 31,
 'stool': 32,
 'table': 33,
 'tent': 34,
 'toilet': 35,
 'tv_stand': 36,
 'vase': 37,
 'wardrobe': 38,
 'xbox': 39}

In [7]:
from torch.utils.data import DataLoader
import torch
from mayavi import mlab


# 指定你想要获取的类别名称
desired_classes = ['airplane','bathtub']  # 例如，获取'chair'和'table'两个类别的数据

# 创建一个列表，用于存储符合指定类别的数据点的索引
selected_indices = []
for idx, (shape_name, _) in enumerate(TRAIN_DATASET.datapath):
    if shape_name in desired_classes:
        selected_indices.append(idx)
class_subset_dataset = torch.utils.data.Subset(TRAIN_DATASET, selected_indices)
batch_size = 32  # 设置批处理大小
dataloader = DataLoader(class_subset_dataset, batch_size=batch_size, shuffle=True)
for batch in dataloader:
    print(batch[0].shape)
    point_cloud = batch[0][0]

    # 创建Mayavi场景
    mlab.figure()
    
    # 使用points3d函数可视化点云
    mlab.points3d(point_cloud[:, 0], point_cloud[:, 1], point_cloud[:, 2], mode="point", color=(0, 0, 1), scale_factor=0.1)
    
    # 显示Mayavi场景
    mlab.show()
    break


torch.Size([32, 10000, 3])


In [16]:
points_list = []
with open('./modelnet40_normal_resampled/modelnet40_shape_names.txt', 'r') as file:
    for line in file:
        points_list.append(line.strip())
print(points_list)

['airplane', 'bathtub', 'bed', 'bench', 'bookshelf', 'bottle', 'bowl', 'car', 'chair', 'cone', 'cup', 'curtain', 'desk', 'door', 'dresser', 'flower_pot', 'glass_box', 'guitar', 'keyboard', 'lamp', 'laptop', 'mantel', 'monitor', 'night_stand', 'person', 'piano', 'plant', 'radio', 'range_hood', 'sink', 'sofa', 'stairs', 'stool', 'table', 'tent', 'toilet', 'tv_stand', 'vase', 'wardrobe', 'xbox']


In [8]:
from pointnet_util import farthest_point_sample, pc_normalize
import torch
point_set = torch.randn(size=(500, 10000, 3))
print(point_set.shape)
point_set = farthest_point_sample(point_set, 1024)

torch.Size([500, 10000, 3])


In [9]:
point_set.shape

torch.Size([500, 1024])

In [1]:
import os

# 获取目录A下的所有文件夹
folders = [f for f in os.listdir('./modelnet40_normal_resampled') if os.path.isdir(os.path.join('./modelnet40_normal_resampled', f))]

# 统计每个文件夹中的txt文件数量
folder_file_counts = {}
for folder in folders:
    folder_path = os.path.join('./modelnet40_normal_resampled', folder)
    txt_files = [f for f in os.listdir(folder_path) if f.endswith('.txt')]
    folder_file_counts[folder] = len(txt_files)

# 按文件数量排序文件夹
sorted_folders = sorted(folder_file_counts.items(), key=lambda x: -x[1])

# 输出排序后的文件夹及其文件数量
# for folder, file_count in sorted_folders:
#     print(f"文件夹: {folder}, 文件数量: {file_count}")
ans = []
for i, x in enumerate(sorted_folders):
    # print("'" + x[0] + "': " + str(i) + ",")
    print(x[0])

chair
sofa
airplane
bookshelf
bed
vase
monitor
table
toilet
bottle
mantel
tv_stand
plant
piano
car
desk
dresser
night_stand
glass_box
guitar
range_hood
bench
cone
tent
flower_pot
laptop
keyboard
curtain
bathtub
sink
lamp
stairs
door
radio
xbox
stool
person
wardrobe
cup
bowl
